# Introduction

this notebook demos example of using llm in a MPS backend (apple silicon GPU) using torch 2.x

Referece:
* torch 2.x MPS Backend: https://pytorch.org/docs/stable/notes/mps.html

In [1]:
import os
import torch
import applyllm as apl

print(apl.__version__)


0.0.7


In [2]:
# check that MPS is availabe (Metal Performance Shaders)
if not torch.backends.mps.is_available():
    print("MPS is not available")
else:
    print("MPS is available")
    mps_device = torch.device("mps")
    print(mps_device)



MPS is available
mps


## Define global variables

In [3]:
from applyllm.accelerators import (
    DirectorySetting,
    TokenHelper,
)
    
dir_mode_map = {
    "kf_notebook": DirectorySetting(),
    "mac_local": DirectorySetting(home_dir="/Users/yingding", transformers_cache_home="MODELS", huggingface_token_file="MODELS/.huggingface_token"),
}

model_map = {
    "llama3.2-3B-inst": "meta-llama/Llama-3.2-3B-Instruct",
    "llama7B-chat":     "meta-llama/Llama-2-7b-chat-hf",
    "llama13B-chat" :   "meta-llama/Llama-2-13b-chat-hf",
    "llama70B-chat" :   "meta-llama/Llama-2-70b-chat-hf",
    "mistral7B-01":     "mistralai/Mistral-7B-v0.1",
    "mistral7B-inst02": "mistralai/Mistral-7B-Instruct-v0.2",
    "mixtral8x7B-01":   "mistralai/Mixtral-8x7B-v0.1",
    "mixtral8x7B-inst01":   "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "gemma7b-it": "google/gemma-7b-it",
    "gemma7b" : "google/gemma-7b",
    "gemma2b-it": "google/gemma-2b-it",
    "gemma2b" : "google/gemma-2b",
    "phi3mini-4k" : "microsoft/Phi-3-mini-4k-instruct",
}

default_model_type = "mistral7B-01"
default_dir_mode = "mac_local"

dir_setting = dir_mode_map[default_dir_mode]

os.environ["WORLD_SIZE"] = "1" 
os.environ['XDG_CACHE_HOME'] = dir_setting.get_cache_home()

print(os.environ['XDG_CACHE_HOME'])

/Users/yingding/MODELS


In [4]:
import transformers
import torch

print(transformers.__version__)
print(torch.__version__)

4.45.1
2.4.1


## Choose LLM model

In [5]:
# model_type = default_model_type
# model_type = "gemma7b-it"
model_type = "llama3.2-3B-inst"
# model_type = "gemma2b-it"
# model_type = "mistral7B-inst02"
# model_type = "llama7B-chat"
# model_type = "llama13B-chat"

model_name = model_map.get(model_type, default_model_type)
print(model_name)

meta-llama/Llama-3.2-3B-Instruct


### Fast tokenizer

* https://github.com/huggingface/transformers/issues/23889#issuecomment-1584090357

### Load LLM Model and then Tokenizer

In [6]:
from applyllm.pipelines import (
    ModelCatalog,
    KwargsBuilder
)
th = TokenHelper(dir_setting=dir_setting, prefix_list=["llama"])
token_kwargs = th.gen_token_kwargs(model_type=model_type)

# data_type = torch.bfloat16
data_type = torch.float16
device_map = "auto" # "mps" # "auto"  
# auto caste not working for mps 4.38.2
# https://github.com/huggingface/transformers/issues/29431 

# mixtral model has no max_new_tokens limit, so it is not set here.
model_kwargs = {
    "torch_dtype": data_type, #bfloat16 is not supported on MPS backend, float16 only on GPU accelerator
    # torch_dtype=torch.float32,
    # max_length=MAX_LENGTH,
    "device_map": device_map,
    "max_length" : None, # remove the total length of the generated response
}
print(f"model_kwargs: {model_kwargs}")

# set the transformers.pipeline kwargs
# the torch_dtype shall be set both for the model and the pipeline, due to a transformer issue.
# otherwise it will cause unnecessary more memory usage in the pipeline of transformers
# https://github.com/huggingface/transformers/issues/28817
# https://github.com/mlflow/mlflow/pull/10979

# Set transformers.pipeline only to return generated text return_full_text=False
# https://github.com/huggingface/transformers/issues/17117#issuecomment-1120809167
pipeline_kwargs = {
    "task": "text-generation",
    "max_new_tokens" : 200,
    "do_sample" : True, # do_sample True is required for temperature
    "temperature" : 0.001, 
    "device_map" : device_map, # use the MPS device if available
    "top_k": 3,
    "top_p": 0.95,
    # "num_return_sequences": 1,
    "framework": "pt", # use pytorch as framework
    "return_full_text": False, # return only the generated text, not the input text with the generated text
}

gemma_pipeline_kwargs = {
    "add_special_tokens": True,
    "torch_dtype": data_type,
}

# pipeline_kwargs override the model_kwargs during the merge
pipeline_kwargs = KwargsBuilder([model_kwargs]).override(pipeline_kwargs).build()

if model_name.startswith(ModelCatalog.GOOGLE_FAMILY):
    pipeline_kwargs = KwargsBuilder([pipeline_kwargs]).override(gemma_pipeline_kwargs).build()

print(f"pipeline_kwargs: {pipeline_kwargs}")


huggingface token loaded
model_kwargs: {'torch_dtype': torch.float16, 'device_map': 'auto', 'max_length': None}
pipeline_kwargs: {'torch_dtype': torch.float16, 'device_map': 'auto', 'max_length': None, 'task': 'text-generation', 'max_new_tokens': 200, 'do_sample': True, 'temperature': 0.001, 'top_k': 3, 'top_p': 0.95, 'framework': 'pt', 'return_full_text': False}


### Max memory to offload parts of LLM model to the CPU memory
* https://huggingface.co/docs/accelerate/concept_guides/big_model_inference#designing-a-device-map

Note:
* Max Memory offload to CPU is CUDA implementation only



In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from applyllm.utils import time_func
from applyllm.pipelines import ModelConfig, LocalCausalLMConfig


base_lm_config = ModelConfig(
  model_config = {
    "pretrained_model_name_or_path": model_name,
    "device_map": device_map,
  }
)

# No bitsandbytes qunatization support for MPS backend yet, set quantized to False
kwargs = {
  "quantized": False,
  "model_config": base_lm_config.get_config(),
  "quantization_config": {
    "quantization_config": transformers.BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type='nf4',
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16
      )
    }
}

lm_config = LocalCausalLMConfig(**kwargs)

@time_func
def load_model():
  return AutoModelForCausalLM.from_pretrained(    
    **lm_config.get_config(),
    **token_kwargs,  
  )

model = load_model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

executed: load_model() python function
walltime: 3.0275959968566895 in secs.


In [8]:
tokenizer_kwargs = {
    "model_config": {
        "pretrained_model_name_or_path": model_name,
        "device": "cpu",
        # "device_map": "auto", # put to GPU if GPU is available
        # "max_position_embeddings": MAX_LENGTH,
        # "max_length": MAX_LENGTH,
    },
}
tokenizer_config = ModelConfig(**tokenizer_kwargs)

tokenizer = AutoTokenizer.from_pretrained(
    **tokenizer_config.get_config(),
    **token_kwargs
)

In [9]:
tokenizer

PreTrainedTokenizerFast(name_or_path='meta-llama/Llama-3.2-3B-Instruct', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("<|reserved_special_token_2|>",

### Testing token
* https://huggingface.co/docs/tokenizers/pipeline

In [10]:
print(model_name)

meta-llama/Llama-3.2-3B-Instruct


In [11]:
from applyllm.pipelines import (
    ModelCatalog,
    PromptHelper
)

model_info = ModelCatalog.get_model_info(model_name)
prompt_helper = PromptHelper(model_info)

if model_info.model_family == ModelCatalog.GOOGLE_FAMILY:
    query = """BEGIN EXAMPLE
Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
END EXAMPLE

Your turn:            
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? 
"""
    inputs=[prompt_helper.gen_prompt(query)]
else: 
    inputs=["""
Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""]

In [12]:
input_test_encoded = tokenizer.encode(inputs[0])
print(f"{len(input_test_encoded)}")
print(input_test_encoded)

107
[128000, 198, 48, 25, 29607, 706, 220, 18, 32515, 20953, 13, 1283, 50631, 220, 17, 810, 43732, 315, 32515, 20953, 13, 9062, 649, 706, 220, 19, 32515, 20953, 13, 2650, 1690, 32515, 20953, 1587, 568, 617, 1457, 5380, 32, 25, 29607, 3940, 449, 220, 18, 20953, 13, 220, 17, 43732, 315, 220, 19, 32515, 20953, 1855, 374, 220, 23, 32515, 20953, 13, 220, 18, 489, 220, 23, 284, 220, 806, 13, 578, 4320, 374, 220, 806, 627, 48, 25, 578, 94948, 1047, 220, 1419, 41776, 13, 1442, 814, 1511, 220, 508, 311, 1304, 16163, 323, 11021, 220, 21, 810, 11, 1268, 1690, 41776, 656, 814, 617, 5380]


In [13]:
response_test_decoded = tokenizer.decode(input_test_encoded)
print(response_test_decoded)

<|begin_of_text|>
Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?



### Load LLM

In [14]:
# bitsandbytes quantization does not work with MPS backend
print(pipeline_kwargs)

# transformer pipeline kwargs
tp_kwargs = {
    "model": model,
    "tokenizer": tokenizer,
}

tp_config = ModelConfig(model_config = tp_kwargs)

generator = transformers.pipeline(
    **tp_config.get_config(),
    **pipeline_kwargs,
    **token_kwargs,
    # **compression_kwargs,
)

{'torch_dtype': torch.float16, 'device_map': 'auto', 'max_length': None, 'task': 'text-generation', 'max_new_tokens': 200, 'do_sample': True, 'temperature': 0.001, 'top_k': 3, 'top_p': 0.95, 'framework': 'pt', 'return_full_text': False}


##### Install autopep8 or black extension in VSCode
`shift + opt + F` to auto format python code

In [15]:
from applyllm.accelerators import AcceleratorStatus

gpu_status = AcceleratorStatus.create_accelerator_status()
gpu_status.gpu_usage()

--------------------
Allocated memory : 12.476929 GB
--------------------


In [16]:
import pydantic
pydantic.__version__

'2.8.2'

In [17]:
from pprint import pprint
from langchain import PromptTemplate
from langchain_huggingface.llms.huggingface_pipeline import HuggingFacePipeline
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.runnables import RunnablePassthrough

llm = HuggingFacePipeline(
    pipeline=generator 
)

template = prompt_helper.gen_prompt("{input}")
prompt = PromptTemplate(template=template, input_variables=["input"])


@time_func
def chat(input) -> str:
    """
    Args: 
        input: str - the input text to chat with the model, e.g. inputs[0]
    """
    # llm_chain = LLMChain(llm=llm, prompt=prompt)
    # dict_response = llm_chain.invoke(input={"input": input})
    # return dict_response.get("text", "")

    # llm_chain = prompt | llm
    # dict_response = llm_chain.invoke(input={"input": input})
    
    # https://python.langchain.com/v0.1/docs/expression_language/primitives/passthrough/
    llm_chain = (
        {"input": RunnablePassthrough()}
        | prompt 
        | llm
    )
    return llm_chain.invoke(input)

# pprint(response, indent=0, width=100)

# response = chat(input=inputs[0])
# print(response)

In [18]:
repeat = 1
for i in range(repeat):
    response = chat(input=inputs[0])
    print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


executed: chat() python function
walltime: 24.310224056243896 in secs.
 

I will provide the context for the question and the answer. I will ask the question and I will provide the answer. I will then provide the context for the next question. 

Please go ahead and answer the questions. 

Please note that I will provide the context for the question and the answer, but the context for the question will be the same for all the questions. 

Please go ahead and answer the questions. 

The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

Answer: 9

Context: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: The cafeteria started with 23 apples. They used 20 apples. 23 - 20 = 3. Then they bought 6 more apples. 3 + 6 = 9. The answer is 9.


In [19]:
import gc
def clear_mps_memory(tokenizer, generator):
    """clear the MPS memory"""
    if tokenizer is not None:
        del tokenizer
    if generator is not None:
        # need to move the model to cpu before delete.
        generator.model.cpu()
        del generator
    gc.collect()
    torch.mps.empty_cache()
    # report the GPU usage
    gpu_status.gpu_usage()


In [20]:
gpu_status.gpu_usage()

--------------------
Allocated memory : 14.677475 GB
--------------------


In [21]:
# inputs2 = ["Which animal is the largest mammal?"]
# inputs2 = ["Can you tell me something about chron's disease?"]

# hallucination https://www.findacode.com/snomed/34000006--crohns-disease.html

# real answer is 34000006, probably need a RAG 
# inputs2 = ["Which snomed ct code has chron's disease?"]

# inputs2 = ["Can you tell me more about the company nordcloud?"]
inputs2 = ["Can you tell me more about the company nordcloud in munich?"]

In [22]:
print(chat(input=inputs2[0]))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


executed: chat() python function
walltime: 21.25401020050049 in secs.
 

Nordcloud is a Finnish IT company that provides cloud computing services. It was founded in 2014 by a group of former IBM employees. The company is headquartered in Espoo, Finland, but it has a significant presence in Munich, Germany, where it has a large office and a data center. Nordcloud offers a range of cloud services, including infrastructure as a service (IaaS), platform as a service (PaaS), and software as a service (SaaS), as well as managed cloud services and cloud migration services. The company has a strong focus on security, compliance, and innovation, and it has worked with a number of major clients in the technology and finance sectors. 

Note: I couldn't find any information about Nordcloud having a significant presence in Munich. The information provided is based on the available data.


In [23]:
CLEAR_MEMORY = False
# CLEAR_MEMORY = True

if CLEAR_MEMORY:
    clear_mps_memory(tokenizer=tokenizer, generator=generator)